In [1]:
# https://keras.io/
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import ReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 32
num_classes = 10
epochs = 50
num_layer = 5

E:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [3]:
def space_to_depth_x8(x):
    return tf.space_to_depth(x, block_size=8)

In [4]:
# Create a block using 32 (same) > 64 (same) > 128 (same) > 256 (same) > 512 (same) > MP 
def create_block(input, block_no, num_layer = 5):
    temp = input
    layer_name = "block-" + str(block_no) + "_bc"
    BatchNorm = BatchNormalization(name=layer_name)(temp)
    temp = Activation('relu')(BatchNorm)
    num_filter = 32
    one_x_one_filter_id = 1
    # Adding layers 
    for i in range(num_layer):  
        # Adding 1x1 filter to reduce number of channels otherwise number of parameters crosses million
        # and processing becomes very slow
        if (num_filter >= 128) :
            layer_name = "block-" + str(block_no) + "_1_1-" + str(one_x_one_filter_id)
            one_x_one_filter_id += 1
            temp = Conv2D(32, 1, activation='relu', name=layer_name, use_bias=False)(temp)
        layer_name = "block-" + str(block_no) + "_layer" + "-" + str(i +1)
        temp = Conv2D(num_filter, (3,3), name=layer_name, use_bias=False, padding='same', activation='relu')(temp)
        num_filter *= 2
        
    # Adding 1x1 filter to reduce number of channels before applying Maxpool
    layer_name = "block-" + str(block_no) + "_1_1-" + str(one_x_one_filter_id)
    one_x_one_filter_id += 1
    temp = Conv2D(32, 1, activation='relu', name=layer_name, use_bias=False)(temp)
    
    # Adding maxpool
    layer_name = "block-" + str(block_no) + "_maxpool_layer"
    temp = MaxPooling2D(pool_size=(2, 2), name=layer_name)(temp)    
    
    return temp
       

In [5]:
input = Input(shape=(img_height, img_width, channel,))

# Layer 1
layer1 = create_block(input, 1, num_layer)
skip_connection = layer1


# Layer 2
layer2 = create_block(layer1, 2, num_layer)

# Layer 3
layer3 = create_block(layer2, 3,num_layer)

# Layer 4
layer4 = create_block(layer3, 4,num_layer)

# skip_connection: prepare for concatenation by applying 1x1 filter and Lambda on space_to_depth_x8 
skip_connection = BatchNormalization(name="block-5_bc")(skip_connection)
skip_connection = ReLU()(skip_connection)
skip_connection = Lambda(space_to_depth_x8)(skip_connection)

# Concatenate layer4 and skip_connection to get previous output layer and then flatten

prev_output_layer = Concatenate(axis=-1)([layer4,skip_connection])
prev_output_layer = Flatten()(prev_output_layer)

# Output layer

output = Dense(num_classes, activation='softmax')(prev_output_layer)



In [6]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block-1_bc (BatchNormalization) (None, 32, 32, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 3)    0           block-1_bc[0][0]                 
__________________________________________________________________________________________________
block-1_layer-1 (Conv2D)        (None, 32, 32, 32)   864         activation_1[0][0]               
__________________________________________________________________________________________________
block-1_la

In [7]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [8]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 97s 2ms/step - loss: 1.5158 - acc: 0.4656 - val_loss: 1.2432 - val_acc: 0.5674
Epoch 2/50
50000/50000 [==============================] - 81s 2ms/step - loss: 1.1259 - acc: 0.6094 - val_loss: 1.1173 - val_acc: 0.6127
Epoch 3/50
50000/50000 [==============================] - 83s 2ms/step - loss: 0.9697 - acc: 0.6655 - val_loss: 1.1831 - val_acc: 0.6157
Epoch 4/50
50000/50000 [==============================] - 86s 2ms/step - loss: 0.8752 - acc: 0.6986 - val_loss: 1.1363 - val_acc: 0.6467
Epoch 5/50
50000/50000 [==============================] - 85s 2ms/step - loss: 0.8030 - acc: 0.7218 - val_loss: 0.9813 - val_acc: 0.6603
Epoch 6/50
50000/50000 [==============================] - 86s 2ms/step - loss: 0.7425 - acc: 0.7430 - val_loss: 0.9464 - val_acc: 0.6880
Epoch 7/50
50000/50000 [==============================] - 86s 2ms/step - loss: 0.6784 - acc: 0.7628 - val_loss: 1.0170 - val_acc